Implementation using  TensorFlow


In [4]:
import numpy as np
import pandas as pd

In [5]:
census=pd.read_csv("/Users/apekshagyawali/Desktop/deeplearn/census_data.csv")

In [6]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Convert strings as labes to 0s and 1s using .apply()

In [7]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [8]:
def label_fix(label):
    if label=='<=50K':
        return 0
    else:
        return 1

In [9]:
census['income_bracket']=census['income_bracket'].apply(label_fix)

perform train test split

In [10]:
from sklearn.model_selection import train_test_split 

In [11]:
x_data=census.drop('income_bracket', axis=1)
y_labels=census['income_bracket']
x_train, x_test, y_train, y_test=train_test_split(x_data, y_labels, test_size=0.3, random_state=101)

In [12]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [13]:
import tensorflow as tf

2024-02-19 18:13:13.293929: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


create features columns for the categorical values

In [14]:
gender=tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation=tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status=tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship=tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education=tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass=tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country=tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


create feature columns for continuous using numeric_columns

In [15]:
age=tf.feature_column.numeric_column("age")
education_num=tf.feature_column.numeric_column("education_num")
capital_gain=tf.feature_column.numeric_column("capital_gain")
capital_loss= tf.feature_column.numeric_column("capital_loss")
hours_per_week=tf.feature_column.numeric_column("hours_per_week")






Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


put all these variables into a single list 

In [16]:
feat_cols=[gender,occupation, marital_status, relationship, education, workclass, native_country, age, education_num, capital_gain, capital_loss, hours_per_week]

In [17]:
# Create input functions using tf.data.Dataset API
def input_func(features, labels, batch_size=100, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(features))
    dataset = dataset.batch(batch_size)
    return dataset

In [18]:
# Define input function for prediction
def predict_input_fn(features, batch_size=100):
    dataset = tf.data.Dataset.from_tensor_slices(dict(features))
    dataset = dataset.batch(batch_size)
    return dataset

In [19]:
# Create input functions for training and evaluation
train_input_fn = lambda: input_func(x_train, y_train)
eval_input_fn = lambda: input_func(x_test, y_test, shuffle=False)


In [20]:
model=tf.estimator.LinearClassifier(feature_columns=feat_cols)

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/5q/xycqttws0ll8pjcw509y6tmh0000gp/T/tmpb40cu5ay', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'wor

In [21]:
model.train(input_fn=train_input_fn, steps=5000)


Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving chec

2024-02-19 18:13:53.652369: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype string and shape [22792]
	 [[{{node Placeholder/_8}}]]
2024-02-19 18:13:53.653151: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [22792]
	 [[{{node Placeholder/_0}}]]


Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 221.602
INFO:tensorflow:loss = 3.1567229e-06, step = 100 (0.451 sec)
INFO:tensorflow:global_step/sec: 643.956
INFO:tensorflow:loss = 5.6925537e-06, step = 200 (0.156 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 228...
INFO:tensorflow:Saving checkpoints for 228 into /var/folders/5q/xycqttws0ll8pjcw509y6tmh0000gp/T/tmpb40cu5ay/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 228...
INFO:tensorflow:Loss for final step: 4.5574902e-06.


create prediction input function

In [22]:
pred_input_fn = lambda: predict_input_fn(x_test)


In [23]:
predictions=list(model.predict(input_fn=pred_input_fn))

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/5q/xycqttws0ll8pjcw509y6tmh0000gp/T/tmpb40cu5ay/model.ckpt-228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2024-02-19 18:14:11.562945: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [9769]
	 [[{{node Placeholder/_4}}]]


In [24]:
predictions[0]

{'logits': array([20.757364], dtype=float32),
 'logistic': array([1.], dtype=float32),
 'probabilities': array([9.6647634e-10, 9.9999994e-01], dtype=float32),
 'class_ids': array([1]),
 'classes': array([b'1'], dtype=object),
 'all_class_ids': array([0, 1], dtype=int32),
 'all_classes': array([b'0', b'1'], dtype=object)}

create a list of only the class_ids  key values from the prediction list of dictionaries

In [32]:
final_preds=[]
for pred in predictions:
    final_preds.append(pred['class_ids'][0])
    
    

In [33]:
final_preds[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [34]:
from sklearn.metrics import classification_report


In [35]:
print(classification_report(y_test, final_preds))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      9769

    accuracy                           1.00      9769
   macro avg       1.00      1.00      1.00      9769
weighted avg       1.00      1.00      1.00      9769

